In [5]:
import os

# Edit this to place the new inputs:
expt_input_dir = "../expt-inputs-001"
os.makedirs(expt_input_dir, exist_ok=True)

# Edit this to set the input template.  This is the bulk of the inputs file.
default_f_base = "input_8_v2"
default_f = f"../data-sets/{default_f_base}.bay"
default_f_output = "output_8_v2"

# Edit this to point to your data directory
data_directory = "/global/cfs/cdirs/m5071/sfw/ExaEpi-mpich/data"
census_file = "CensusData/BayArea.dat"
workflow_file = "CensusData/BayArea-wf.bin"
case_file = "CaseData/Feb1.cases"

# These are the parameter combinations that we are setting uniquely for each run
#       looped over by the loops below and ultimately by the workflow:
dyn_params = ["agent.seed", "disease.p_trans"]

# Edit this to point to the  that you want to use
census_line = f'agent.census_filename = {data_directory}/{census_file}\n'
agent_workerflow_line= f'agent.workerflow_filename = {data_directory}/{workflow_file}\n'
disease_case_line = f'disease.case_filename = {data_directory}/{case_file}\n'

line_params = ["agent.census_filename", "agent.workerflow_filename", "disease.case_filename"]

lines = [census_line, agent_workerflow_line, disease_case_line]
with open(default_f) as fin:
    for line in fin:
        idx = line.find("=")
        if idx != -1 and not line.startswith("#"):
            param = line[:idx].strip()
            if param not in dyn_params and param not in line_params:
                lines.append(line)

with open(expt_input_dir + "/input_map.csv", "w") as mout:
    mout.write("fname,seed,p_trans\n")
    i = 1
    for seed in range(1, 51):
        for p_trans in [0.0001, 0.00015, 0.0002, 0.00025, 0.0003, 0.00035, 0.0004, 0.00045, 0.0005, 0.00055]:
            fname = f"{default_f_base}_{i}.bay"
            fpath = os.path.join(expt_input_dir, fname)
            output_fname = f"{default_f_output}_{i}.dat"
            mout.write(f"{fname},{seed},{p_trans}\n")
            with open(fpath, "w") as iout:
                iout.writelines(lines)
                iout.write(f"agent.seed = {seed}\n")
                iout.write(f"disease.p_trans = {p_trans}\n")
                iout.write(f"diag.output_filename = {output_fname}\n")
            
            i += 1
